# How to use the invenio API

- first run:

  ```bash
  export PYTHON=/usr/bin/python3.12
  ./nrp upgrade
  ./nrp develop
  ```

- run API with:

  ```bash
  ./run.sh
  ```

- grant user access (doesn't work):

  ```bash
  source .venv/bin/activate
  invenio access allow admin-access user test@test.com
  ```

## Useful links

- [Invenio API requests](https://invenio.readthedocs.io/en/latest/getting-started/quickstart/crud-operations.html)
- [Invenio REST API](https://inveniordm.docs.cern.ch/reference/rest_api_index/) (they are lying about authentication only with Access Tokens, you can authenticate as shown below)
- [NRP Docs](https://nrp-cz.github.io/docs)
- [NRP models](https://narodni-repozitar.github.io/developer-docs/docs/technology/invenio/nrp-toolchain/edit-metadata)

## Actions

In [12]:
import requests
import json

API_ENDPOINT = 'https://mdrepo.eu/api'
# API_ENDPOINT = 'https://inveniordm.web.cern.ch/api'

# test if API is up
response = requests.get(f'{API_ENDPOINT}/login')
if response.status_code != 405:
    raise Exception('API returned with code', response.status_code)

### Login

In [13]:
data = {
    'email': 'test@test.com',
    'password': '123456'
}

response = requests.post(
    f'{API_ENDPOINT}/login',
    data=data
)
print(response.json())

session_cookie = response.cookies['session']
print(response.cookies)
print('Session cookie:', session_cookie)

{'confirmed_at': '2024-10-13T16:51:31.314348', 'email': 'test@test.com', 'id': 1, 'last_login_at': '2024-10-13T17:05:11.649876', 'roles': []}
<RequestsCookieJar[<Cookie session=5c1807a164cdfc54_670bfdc7.p6z5aaXgUYpg9y8LNJQ-eUBmiw8 for mdrepo.eu/>]>
Session cookie: 5c1807a164cdfc54_670bfdc7.p6z5aaXgUYpg9y8LNJQ-eUBmiw8


### Search

In [49]:
params = {
    'q': 'LRIPCCPVNLKRLLVVVVVVVLVVVVIVGALLMGL',
    'size': 10,
    'page': 1
}
response = requests.get(f'{API_ENDPOINT}/experiments', params=params)
print(response.json())

{'hits': {'hits': [{'id': 'k3b56-3yg30', 'created': '2024-10-12T18:01:18.730213+00:00', 'updated': '2024-10-12T18:01:18.829993+00:00', 'links': {'applicable-requests': 'https://mdrepo.eu/api/experiments/k3b56-3yg30/requests/applicable', 'draft': 'https://mdrepo.eu/api/experiments/k3b56-3yg30/draft', 'files': 'https://mdrepo.eu/api/experiments/k3b56-3yg30/files', 'latest': 'https://mdrepo.eu/api/experiments/k3b56-3yg30/versions/latest', 'latest_html': 'https://mdrepo.eu/experiments/k3b56-3yg30/latest', 'publish': 'https://mdrepo.eu/api/experiments/k3b56-3yg30/draft/actions/publish', 'record': 'https://mdrepo.eu/api/experiments/k3b56-3yg30', 'requests': 'https://mdrepo.eu/api/experiments/k3b56-3yg30/requests', 'self': 'https://mdrepo.eu/api/experiments/k3b56-3yg30', 'self_html': 'https://mdrepo.eu/experiments/k3b56-3yg30', 'versions': 'https://mdrepo.eu/api/experiments/k3b56-3yg30/versions'}, 'revision_id': 2, '$schema': 'local://experiments-1.0.0.json', 'metadata': {'simulations': [{'_d

### Create record

In [21]:
# XXX: session_cookie is required to create a record

cookies = {'session': session_cookie}

with open('demo-data/example_metadata.json', 'r') as f:
    data = json.load(f)

response = requests.post(
    f'{API_ENDPOINT}/experiments',
    cookies=cookies,
    json=data
)

print('Status:', response.status_code)
print('Full response:', response.text)

if 'errors' in response.json():
    print('Errors:', response.json()['errors'])

record_id = response.json().get('id', None)
print('Record ID:', record_id)

Status: 400
Full response: {"status": 400, "message": "A validation error occurred.", "errors": [{"field": "_schema", "messages": ["Default community not defined in input."]}]}
Errors: [{'field': '_schema', 'messages': ['Default community not defined in input.']}]
Record ID: None


### View record

In [45]:
cookies = {'session': session_cookie}

response = requests.get(
    f'{API_ENDPOINT}/experiments/{record_id}/draft',
    cookies=cookies
    )
print(response.json())

{'id': '6130f-yfa30', 'created': '2024-10-12T19:11:15.812467+00:00', 'updated': '2024-10-12T19:11:15.837524+00:00', 'links': {'applicable-requests': 'https://mdrepo.eu/api/experiments/6130f-yfa30/draft/requests/applicable', 'draft': 'https://mdrepo.eu/api/experiments/6130f-yfa30/draft', 'edit_html': 'https://mdrepo.eu/experiments/6130f-yfa30/edit', 'files': 'https://mdrepo.eu/api/experiments/6130f-yfa30/draft/files', 'latest': 'https://mdrepo.eu/api/experiments/6130f-yfa30/versions/latest', 'latest_html': 'https://mdrepo.eu/experiments/6130f-yfa30/latest', 'publish': 'https://mdrepo.eu/api/experiments/6130f-yfa30/draft/actions/publish', 'record': 'https://mdrepo.eu/api/experiments/6130f-yfa30', 'requests': 'https://mdrepo.eu/api/experiments/6130f-yfa30/draft/requests', 'self': 'https://mdrepo.eu/api/experiments/6130f-yfa30/draft', 'self_html': 'https://mdrepo.eu/experiments/6130f-yfa30/preview', 'versions': 'https://mdrepo.eu/api/experiments/6130f-yfa30/versions'}, 'revision_id': 3, '$

### List files of a record

In [10]:
cookies = {'session': session_cookie}

response = requests.get(f'{API_ENDPOINT}/experiments/{record_id}/draft/files', cookies=cookies)
print(response.json())

{'enabled': True, 'links': {'self': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files'}, 'entries': [{'key': 'SPC.tpr', 'storage_class': 'L', 'checksum': 'md5:eb43ff185b5e6bdc3a2d0faea2e1db19', 'size': 5303276, 'created': '2024-10-13T15:50:39.598743+00:00', 'updated': '2024-10-13T15:50:40.767565+00:00', 'status': 'completed', 'mimetype': 'application/octet-stream', 'version_id': 'ab91bbde-15c8-4f3c-a7b4-66625e4ac88e', 'file_id': '781b06a8-57ae-4d52-8c57-3e0db79f566b', 'bucket_id': 'c33a1b14-6670-4f19-97a4-5df49e287888', 'metadata': None, 'access': {'hidden': False}, 'links': {'commit': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files/SPC.tpr/commit', 'content': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files/SPC.tpr/content', 'preview': 'https://mdrepo.eu/experiments/ydn5b-hn705/files/SPC.tpr/preview', 'self': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files/SPC.tpr'}}], 'default_preview': None, 'order': []}


### Upload file to a record

In [9]:
local_file_name = 'demo-data/SPC.tpr'
remote_file_name = 'SPC.tpr'

cookies = {'session': session_cookie}

# set file metadata
data = [{'key': remote_file_name}]
response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/files',
    cookies=cookies,
    json=data
)
print('POST metadata:', response.text)

# upload file
with open(local_file_name, 'rb') as f:
    response = requests.put(
        f'{API_ENDPOINT}/experiments/{record_id}/draft/files/{remote_file_name}/content',
        cookies=cookies,
        data=f,
        stream=True
    )
    print(response.status_code)
    print('PUT file:', response.text)

# commit file
response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/files/{remote_file_name}/commit',
    cookies=cookies
)
print('POST commit:', response.text)


POST metadata: {"enabled": true, "links": {"self": "https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files"}, "entries": [{"key": "SPC.tpr", "created": "2024-10-13T15:50:39.598743+00:00", "updated": "2024-10-13T15:50:39.604484+00:00", "status": "pending", "metadata": null, "access": {"hidden": false}, "links": {"commit": "https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files/SPC.tpr/commit", "content": "https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files/SPC.tpr/content", "preview": "https://mdrepo.eu/experiments/ydn5b-hn705/files/SPC.tpr/preview", "self": "https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files/SPC.tpr"}}], "default_preview": null, "order": []}
200
PUT file: {"key": "SPC.tpr", "created": "2024-10-13T15:50:39.598743+00:00", "updated": "2024-10-13T15:50:39.604484+00:00", "status": "pending", "metadata": null, "access": {"hidden": false}, "links": {"commit": "https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/files/SPC.tpr/commit", "content": "https://mdr

### Download file from a record

In [37]:
cookies = {'session': session_cookie}
remote_file_name = 'SPC.tpr'

response = requests.get(
    f'{API_ENDPOINT}/experiments/{record_id}/files/{remote_file_name}/content',
    cookies=cookies
)
if response.status_code >= 400:
    print(response.text)
    raise Exception('Failed to download file')

with open(remote_file_name, 'wb') as f:
    f.write(response.content)

### Publish record

In [11]:
cookies = {'session': session_cookie}

response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/actions/publish',
    cookies=cookies
)
print('POST publish:', response.json())

POST publish: {'id': 'ydn5b-hn705', 'created': '2024-10-13T15:51:06.781358+00:00', 'updated': '2024-10-13T15:51:06.891864+00:00', 'links': {'applicable-requests': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/requests/applicable', 'draft': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/draft', 'files': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/files', 'latest': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/versions/latest', 'latest_html': 'https://mdrepo.eu/experiments/ydn5b-hn705/latest', 'publish': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/draft/actions/publish', 'record': 'https://mdrepo.eu/api/experiments/ydn5b-hn705', 'requests': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/requests', 'self': 'https://mdrepo.eu/api/experiments/ydn5b-hn705', 'self_html': 'https://mdrepo.eu/experiments/ydn5b-hn705', 'versions': 'https://mdrepo.eu/api/experiments/ydn5b-hn705/versions'}, 'revision_id': 2, '$schema': 'local://experiments-1.0.0.json', 'metadata': {'simulations': [{'_dump_s